# Aplicaciones de Ciencias de la Computación (INF265)
## Laboratorio 1: Agente aspirador de 6 posiciones

Al finalizar el presente laboratorio se debe tener implementado el entorno de trabajo del agente aspirador de 6 posiciones y un programa reflexivo simple para dicho agente. Las posiciones del entorno son denotadas como loc_A, loc_B, loc_C, loc_D, loc_E y loc_F. Cada una de estas posiciones puede tener el estado 'Dirty' o 'Clean'.  
Al final del notebook deberas responder a las preguntas planteadas. 

# Clase <b>Thing</b>

Esta clase generica representa cualquier objeto fisico que puede aparecer en un <b>Ambiente</b>. (No editar)  

In [12]:
class Thing(object):

    def is_alive(self):
        """Cosas 'vivas'deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")

# Clase <b>Agent</b>

Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la funcion que implementa el <b>programa del agente</b>. Esta funcion debe tomar como argumento la <b>percepcion</b> del agente y debe retornar una <b>accion</b>. La definicion de Percepcion y Accion depende del ambiente de trabajo (environment) donde el agente existe. El agente tambien puede tener el slot <b>.performance</b>, que mide el desempeño del agente en su ambiente.

In [13]:
import collections
import random

class Agent(Thing):

    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.Callable)
        self.program = program

# Clase <b>Environment</b>

Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno tipicamente se necesitará implementar 2 cosas:
<b>percept</b>, que define la percepción que el agente ve; y 
<b>execute_action</b>, que define los efectos de ejecutar una acción. 
El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location. (No editar)

In [14]:
class Environment(object):

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepción que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una acción 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localización por defecto para colocar una nueva cosa sin localización especificada."""
        return None

    def is_done(self):
        """Retorna True si no hay ningón agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Añade una cosa thing al entorno en la localización location. 
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "No añade la misma cosa dos veces"
        thing.location = location if location is not None else self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta). """
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

#### Clase <b>VacuumEnvironment</b>

Esta clase implementa el entorno del aspirador de 6 posiciones: loc_A, loc_B, loc_C, loc_D, loc_E y loc_F. Cada una de estas posiciones puede tener el estado 'Dirty' o 'Clean'. Un agente en este entorno percibe su localizacion y el estado de la misma

In [15]:

# Considere el siguiente ambiente:

#############
# A # B # C #
#############
# D # E # F #
#############
# Complete las posiciones reemplazando los guiones bajos por números
loc_A, loc_B, loc_C, loc_D, loc_E, loc_F = (0,0), (0,1), (0, 2), (1,0), (1,1), (1,2)

class VacuumEnvironment(Environment):

    def __init__(self):
        super().__init__()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty']),
                       loc_C: random.choice(['Clean', 'Dirty']),
                       loc_D: random.choice(['Clean', 'Dirty']),
                       loc_E: random.choice(['Clean', 'Dirty']),
                       loc_F: random.choice(['Clean', 'Dirty']),}

    def thing_classes(self):
        return [ReflexVacuumAgent]

    def percept(self, agent):
        """Retorna la posición del agente y el estado de la posición (Dirty/Clean)."""
        return (agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Implementa el MAPA DE TRANSICIÓN: Cambia la posición del agente y/o el estado de la posición; 
        Cada aspiración (acción 'suck') en una localización Dirty provoca un aumento de desempeño en 10 unidades;
        Cada movida efectiva Right, Left, Up o Down provoca una disminución de desempeño en 1 unidad """
        
        if action == 'Suck':
            self.status[agent.location] = 'Clean'
            agent.performance += 10
        else:
            # Completar los movimientos para el agente
            # Debes identificar la direccion de la accion y modificar acordemente el x o y
            # Pista: Ten cuidado con los limites del ambiente!
            actual_x, actual_y = agent.location
            # Complete su codigo aqui
            if action=='Left':
                if agent.location==loc_B or agent.location==loc_C or agent.location==loc_E or agent.location==loc_F:
                    actual_y-=1
            elif action=='Right':
                if agent.location==loc_A or agent.location==loc_B or agent.location==loc_D or agent.location==loc_E:
                    actual_y+=1
            if action=='Up':
                if agent.location==loc_D or agent.location==loc_F or agent.location==loc_E:
                    actual_x-=1
            elif action=='Down':
                if agent.location==loc_C or agent.location==loc_A or agent.location==loc_B:
                    actual_x+=1 
            
            
            agent.location = actual_x, actual_y
            agent.performance -= 1

    def default_location(self, thing):
        """Devuelve una posicion aleatoria."""
        return random.choice([loc_A, loc_B, loc_C, loc_D])

# Agente Aspirador de 6 posiciones con Programa Reactivo Simple

Este agente es el agente aspirador de cuatro posiciones que usa un programa reactivo simple: realiza una accion basado en la percepción (posicion, estado) actual

In [16]:
def ReflexVacuumAgent():
    
    def program(percept):
        location, status = percept
        if status == 'Dirty':
            return 'Suck'
        elif location == loc_A:
            return 'Right'
        elif location == loc_B:
            return 'Right'
        elif location == loc_C:
            return 'Down'
        elif location == loc_D:
            return 'Up'
        elif location == loc_E:
            return 'Left'
        elif location == loc_F:
            return 'Left'
    return Agent(program)

# Probando el agente reflexivo simple en su entorno

In [17]:
"""Crea el entorno del aspirador de 6 posiciones con 2 posiciones en estado 'Dirty'"""
e = VacuumEnvironment()
e.status = {loc_A: 'Clean',  loc_B: 'Dirty', loc_C: 'Clean', loc_D: 'Dirty', loc_E: 'Clean', loc_F: 'Clean'}

"""Crea un agente reflexivo simple"""
a = ReflexVacuumAgent()

"""Añade el agente creado al entorno en posicion loc_A"""
e.add_thing(a, location=loc_A) 

# Imprime el estado inicial del ambiente y localizacion del agente
print("Estado Inicial del Ambiente: {}".format(e.status))
print("ReflexVacuumAgent esta localizado en {} con desempeño = {}".format(a.location, a.performance))

"""Ejecuta el entorno 6 pasos y obtiene el desempeño del agente"""
e.run(6)

# Imprime el estado actual del ambiente, localizacion del agente y su desempeño
print("Estado del Ambiente despues de 6 pasos: {}".format(e.status))
print("ReflexVacuumAgent esta localizado en {} con desempeño = {}".format(a.location, a.performance))


Estado Inicial del Ambiente: {(0, 0): 'Clean', (0, 1): 'Dirty', (0, 2): 'Clean', (1, 0): 'Dirty', (1, 1): 'Clean', (1, 2): 'Clean'}
ReflexVacuumAgent esta localizado en (0, 0) con desempeño = 0
Estado del Ambiente despues de 6 pasos: {(0, 0): 'Clean', (0, 1): 'Clean', (0, 2): 'Clean', (1, 0): 'Dirty', (1, 1): 'Clean', (1, 2): 'Clean'}
ReflexVacuumAgent esta localizado en (1, 0) con desempeño = 5


# Preguntas:
1. Completar el codigo en la clase Vacuum Enviroment. No cambiar la medida de desempeño que tiene el ambiente, sólo completar la lógica de movimientos. (**4 pts**)
2. Después de haber completado el código, ¿puede plantear un programa de Agente que maximice de mejor manera  la medida de desempeño implementada en el entorno?  No es necesario modificar el código, sino que puede explicarlo en sus propias palabras. (4 pts) (**4 pts**)
3. Basado en el programa de agente original, ¿que debería cambiar en el entorno para que el programa de agente maximize la medida de desempeño? Modifique el código. (**4 pts**)
4. Según el entorno, ¿existe la posibilidad de que el agente obtenga un desempeño sea negativo? En caso de que su respuesta sea positiva, explique como sería dicho escenario. (**4 pts**)
5. ¿Qué característica tendria que tener el entorno para necesitar un agente reactivo basado en modelo? (**4 pts**) 

In [7]:
# Pregunta 1: Codigo completado en la clase Vacuum Enviroment.

In [8]:
# Pregunta 2: 
# La unica forma en la cual aumenta la medida de desempeño es aspirar basura dado que suma +10 en caso lo haga, es por ello
# que en esta parte no se necesitaran los movimientos que realice la aspiradora, solo se necesita retornar suck siempre. 
# En la funcion program solo se tendria que hacer un action='Suck' y retornarla. AL hacer esto la aspiradora siempre
# estará limpiando sin importar si el estado es 'Dirty' o 'Clean' entonces se estaría maximizando su desempeño dado que siempre
# estará sumando 10. 
# Ejem: Para un caso de e.run(6) o en otras palabras 6 pasos la aspiradora presentaría un desempeño de 60.

In [9]:
#Pregunta 3:
#Cambiaria la funcion execute_action de VacuumEnvironment para que maximice la medida de desempeño pero 
#de forma que solo aspire cuando el espacio esta sucio. De esta forma conseguimos premiar la suciedad 
#efectiva.
#CODIGO:

#EL CAMBIO SE REALIZO EN LA LINEA 6 donde agregamos que la condicion de aspirar solo cuando la ubicacion del agente
#este sucia.

#1 def execute_action(self, agent, action):
#2         """Implementa el MAPA DE TRANSICIÓN: Cambia la posición del agente y/o el estado de la posición; 
#3         Cada aspiración (acción 'suck') en una localización Dirty provoca un aumento de desempeño en 10 unidades;
#4         Cada movida efectiva Right, Left, Up o Down provoca una disminución de desempeño en 1 unidad """
#5         
#6         if action == 'Suck' and self.status[agent.location]=='Dirty':
#7             self.status[agent.location] = 'Clean'
#8             agent.performance += 10
#9         else:
#10            # Completar los movimientos para el agente
#11            # Debes identificar la direccion de la accion y modificar acordemente el x o y
#12            # Pista: Ten cuidado con los limites del ambiente!
#13           actual_x, actual_y = agent.location
#14            # Complete su codigo aqui
#15            if action=='Left':
#16                if agent.location==loc_B or agent.location==loc_C or agent.location==loc_E or agent.location==loc_F:
#17                    actual_y-=1
#18            elif action=='Right':
#19                if agent.location==loc_A or agent.location==loc_B or agent.location==loc_D or agent.location==loc_E:
#20                    actual_y+=1
#21            if action=='Up':
#22                if agent.location==loc_D or agent.location==loc_F:
#23                    actual_x-=1
#24            elif action=='Down':
#25                if agent.location==loc_C or agent.location==loc_A:
#26                    actual_x+=1          
#27            agent.location = actual_x, actual_y
#28            agent.performance -= 1

In [10]:
# Pregunta 4:
# Un desempeño negativo se puede dar por dos factores: la cantidad de suciedad en el ambiente y el numero de pasos digitado
# por el usuario que dará el agente.

# CASO 1:
# No existe suciedad en el ambiente y el numero de pasos es mayor a cero

# CASO 2:
# Existe suciedad en el ambiente pero el numero de pasos es menor que el necesario para encontrar la primera suciedad
# Ejemplo: Numero de pasos:4 C-->CLEAN   D-->DIRTY
# A(C) -> B(C) -> C(C)
# |                 | 
# D(D) <- E(C) <- F(C)
# En este caso no encontramos suciedad alguna por lo tanto la medida de desempeño sera negativa.

In [11]:
# Pregunta 5:
# La diferente entre el reactivo basado en modelo y el reactivo simple es que en el de modelo el estado es una 
# variable interna que la modela el agente. El agente no confia mucho en la percepcion actual sino que tiene un estado
# interno el cual actualiza cuando recibe una percepcion pero no solo basado en esta sino en el modelo del mundo.
# La caracteristica ideal para usar esta estructura de agente es que el entorno sea parcialmente observable ya que 
# como lo mencioné anteriormente, el modelo se presta para trabajar en estos entornos.